In [1]:
import pandas as pd


def reformat(df):
    cols = ["Below 0.4","Above 0.9", "Between 0.4 and 0.9", "Full dataset"]
    df = df.reset_index().rename(columns={'index': 'method'})

    for col in cols:
        col_df = df[col].apply(pd.Series)
        col_df = col_df.rename(columns=lambda x: f"{col}_{x}")
        df = pd.concat([df.drop([col], axis=1), col_df], axis=1)

    
    resp_df = df.responses_score.apply(pd.Series)
    resp_df = resp_df.rename(columns=lambda x: f"Response_{x}")
    df = df.drop(['responses_score'],axis=1)
    # df = pd.concat([df.drop(['responses_score'], axis=1), resp_df], axis=1)
    return df

df = pd.read_json('rm_eval/eval_metrics_q_mse.json',orient='index')
df = reformat(df)

df2 = pd.read_json('rm_eval/eval_metrics_q_.json',orient='index')
df2 = reformat(df2)

df3 = pd.read_json('rm_eval/eval_metrics_q_eos.json',orient='index')
df3 = reformat(df3)

df4 = pd.read_json('rm_eval/eval_metrics_q_aug_and_under.json',orient='index')
df4 = reformat(df4)


df5 = pd.read_json('rm_eval/eval_metrics_q_aug_and_under2.json',orient='index')
df5 = reformat(df5)

df6 = pd.read_json('rm_eval/eval_metrics_q_aug_over_and_under.json',orient='index')
df6 = reformat(df6)


df7 = pd.read_json("rm_eval/eval_metrics_q_aug_over_and_under_un_till16.json",orient='index')
df7 = reformat(df7)


df8 = pd.read_json("rm_eval/eval_metrics_q_aug_and_under_un_till16_1.json",orient='index')
df8 = reformat(df8)

df

,method,Below 0.4_mse,Below 0.4_mae,Above 0.9_mse,Above 0.9_mae,Between 0.4 and 0.9_mse,Between 0.4 and 0.9_mae,Full dataset_mse,Full dataset_mae
0,abs_mse_s_075_quality,0.3158782,0.51607615,0.06420932,0.19708948,0.12091556,0.28012288,0.12887712,0.2870976


In [2]:
final_df = pd.concat([df3,df2,df4,df5,df6,df7,df8])
final_df = final_df.reset_index(drop=True)

for column in final_df.columns:
    # Skip non-numeric columns
    if column != 'method':
        final_df[column] = pd.to_numeric(final_df[column], errors='coerce')  # 'coerce' will set invalid parsing to NaN

final_df = final_df.round(4)
final_df = final_df[final_df["method"] != "abs_logistic_s_075_quality_eos_token"]

mse_vs_logistic = ['abs_logistic_over_and_under_s_75_quality','abs_sig_mse_s_075_quality','abs_sig_mse_aug_and_under_quality','abs_logistic_aug_and_under_quality','abs_sig_mse_aug_over_and_un_till16_w14_for_04_09_quality','abs_logistic_aug_over_and_un_till16_w14_for_04_09_quality']
mse_vs_logistic_df = final_df[final_df["method"].isin(mse_vs_logistic)].reset_index(drop=True)

mse_vs_logistic_df = mse_vs_logistic_df.drop(columns=["Full dataset_mse","Full dataset_mae"])

weightcomparision = ['abs_logistic_aug_and_under_s_05_quality','abs_logistic_aug_and_under_s_075_quality','abs_logistic_over_and_under_s_05_quality','abs_logistic_over_and_under_s_75_quality','abs_logistic_over_and_under_quality','abs_logistic_s_05_quality','abs_logistic_s_075_quality','abs_logistic_quality']
weightcomparision_df = final_df[final_df["method"].isin(weightcomparision)].reset_index(drop=True)


weightcomparision_df

,method,Below 0.4_mse,Below 0.4_mae,Above 0.9_mse,Above 0.9_mae,Between 0.4 and 0.9_mse,Between 0.4 and 0.9_mae,Full dataset_mse,Full dataset_mae
0,abs_logistic_aug_and_under_s_05_quality,0.1052,0.2799,0.0568,0.2080,0.0275,0.1322,0.0403,0.1601
1,abs_logistic_aug_and_under_s_075_quality,0.0721,0.2267,0.0852,0.2640,0.0335,0.1471,0.0467,0.1762
2,abs_logistic_over_and_under_s_05_quality,0.0519,0.1801,0.1195,0.3174,0.0465,0.1754,0.0605,0.2021
3,abs_logistic_over_and_under_s_75_quality,0.0611,0.1999,0.1027,0.2923,0.0415,0.1662,0.0547,0.1927
4,abs_logistic_over_and_under_quality,0.1051,0.2752,0.0534,0.2009,0.0290,0.1351,0.0407,0.1605
5,abs_logistic_s_05_quality,0.0771,0.2348,0.0862,0.2765,0.0293,0.1392,0.0444,0.1736
6,abs_logistic_s_075_quality,0.0591,0.2040,0.1159,0.3205,0.0369,0.1573,0.0536,0.1919
7,abs_logistic_quality,0.1227,0.3099,0.0518,0.2091,0.0222,0.1192,0.0372,0.1538


In [3]:
# Pivot the DataFrame to have multi-level columns suitable for multirow headers
final_df.columns = pd.MultiIndex.from_tuples(
    [tuple(c.split('_')) if '_' in c and c.split('_')[0] != 'Response' else (c, '') for c in final_df.columns]
)

# Pivot the DataFrame to have multi-level columns suitable for multirow headers
mse_vs_logistic_df.columns = pd.MultiIndex.from_tuples(
    [tuple(c.split('_')) if '_' in c and c.split('_')[0] != 'Response' else (c, '') for c in mse_vs_logistic_df.columns]
)

# Pivot the DataFrame to have multi-level columns suitable for multirow headers
weightcomparision_df.columns = pd.MultiIndex.from_tuples(
    [tuple(c.split('_')) if '_' in c and c.split('_')[0] != 'Response' else (c, '') for c in weightcomparision_df.columns]
)

final_df

method Below 0.4          \
                                                            mse     mae   
0                           abs_sig_mse_s_075_quality    0.0618  0.1981   
1             abs_logistic_aug_and_under_s_05_quality    0.1052  0.2799   
2            abs_logistic_aug_and_under_s_075_quality    0.0721  0.2267   
3            abs_logistic_over_and_under_s_05_quality    0.0519  0.1801   
4            abs_logistic_over_and_under_s_75_quality    0.0611  0.1999   
5                 abs_logistic_over_and_under_quality    0.1051  0.2752   
6                           abs_logistic_s_05_quality    0.0771  0.2348   
7                          abs_logistic_s_075_quality    0.0591  0.2040   
8                                abs_logistic_quality    0.1227  0.3099   
9                  abs_logistic_aug_and_under_quality    0.0853  0.2446   
10                  abs_sig_mse_aug_and_under_quality    0.1056  0.2827   
11  abs_logistic_aug_and_under_s_075_for_04_09_qua...    0.0584  0.1913   
12  abs_logistic_aug_over_and_under_w14_for_04_09_...    0.1235  0.3032   
13  abs_sig_mse_aug_over_and_un_till16_s_075_w_14_...    0.0983  0.2693   
14  abs_sig_mse_aug_over_and_un_till16_w14_for_04_...    0.1223  0.3030   
15  abs_logistic_aug_over_and_un_till16_w14_for_04...    0.0889  0.2555   
16                                               rank    0.0633  0.1879   
17  abs_sig_mse_aug_and_un_till16_s_075_w_14_09_qu...    0.1215  0.3023   
18        abs_sig_mse_aug_and_un_till16_s_075_quality    0.0797  0.2358   

   Above 0.9         Between 0.4 and 0.9         Full dataset          
         mse     mae                 mse     mae          mse     mae  
0     0.0964  0.2800              0.0423  0.1652       0.0541  0.1895  
1     0.0568  0.2080              0.0275  0.1322       0.0403  0.1601  
2     0.0852  0.2640              0.0335  0.1471       0.0467  0.1762  
3     0.1195  0.3174              0.0465  0.1754       0.0605  0.2021  
4     0.1027  0.2923              0.0415  0.1662       0.0547  0.1927  
5     0.0534  0.2009              0.0290  0.1351       0.0407  0.1605  
6     0.0862  0.2765              0.0293  0.1392       0.0444  0.1736  
7     0.1159  0.3205              0.0369  0.1573       0.0536  0.1919  
8     0.0518  0.2091              0.0222  0.1192       0.0372  0.1538  
9     0.0786  0.2493              0.0326  0.1455       0.0461  0.1740  
10    0.0593  0.2132              0.0299  0.1360       0.0425  0.1641  
11    0.1277  0.3235              0.0529  0.1880       0.0672  0.2134  
12    0.0415  0.1689              0.0294  0.1357       0.0406  0.1577  
13    0.0576  0.2112              0.0265  0.1296       0.0390  0.1579  
14    0.0441  0.1796              0.0255  0.1262       0.0381  0.1528  
15    0.0645  0.2320              0.0263  0.1301       0.0393  0.1608  
16    0.1347  0.2843              0.0990  0.2598       0.1022  0.2575  
17    0.0483  0.1931              0.0254  0.1272       0.0388  0.1559  
18    0.0806  0.2589              0.0300  0.1390       0.0441  0.1703

In [4]:
def highlight_extreme_values(s):
    '''
    Highlight the maximum in red, the minimum in green, the second lowest in light green, 
    and the third lowest in blue.
    '''
    is_max = s == s.max()
    is_min = s == s.min()
    
    # Sort the values and safely get the second and third lowest
    sorted_values = s.sort_values()
    is_second_lowest = s == sorted_values.iloc[1] if len(s) > 1 else False
    is_third_lowest = s == sorted_values.iloc[2] if len(s) > 2 else False

    return ['background-color: red' if max_val else 
            'background-color: green' if min_val else 
            'background-color: yellow' if (len(s) > 1 and second_lowest) else 
            'background-color: orange' if (len(s) > 2 and third_lowest) else '' 
            for max_val, min_val, second_lowest, third_lowest in zip(is_max, is_min, is_second_lowest, is_third_lowest)]

final_df['diff'] = (final_df["Below 0.4"]["mae"]-final_df["Above 0.9"]["mae"]).abs()
# Apply the highlight function to the DataFrame except for the 'method' column
df_highlighted = final_df.style.apply(highlight_extreme_values, subset=final_df.columns[1:])

df_highlighted

In [5]:

df_highlighted = mse_vs_logistic_df.style.apply(highlight_extreme_values, subset=mse_vs_logistic_df.columns[1:])

df_highlighted



In [6]:
weightcomparision_df['diff'] = (weightcomparision_df["Below 0.4"]["mae"]-weightcomparision_df["Above 0.9"]["mae"]).abs()

df_highlighted = weightcomparision_df.style.apply(highlight_extreme_values, subset=weightcomparision_df.columns[1:])

df_highlighted

In [7]:

# data for comparing the mode manually:

data = [
[["Explain the importance of a balanced diet."],
    ["A balanced diet is crucial for maintaining good health. It ensures that your body gets all the essential nutrients it needs, including vitamins, minerals, proteins, carbohydrates, and fats. This helps in regulating body functions, supporting growth and development, and preventing chronic diseases. A balanced diet also contributes to mental well-being and energy levels, playing a key role in overall lifestyle quality.",
    "Eating a balanced diet means just choosing different foods. It's like, you shouldn't eat only pizza every day. Maybe add some fruits or something sometimes. It's better than eating bad stuff all the time, I guess.",
    "While talking about a balanced diet, it's interesting to note that the agricultural practices of the 18th century significantly differed from modern techniques. Back then, crop rotation and the use of natural fertilizers led to a different quality of produce. This historical perspective, though fascinating, does not directly address the current dietary practices or their health implications.",
    "Diet balance is like a garden salad, twirling in nutrition and sunshine rainbows. Vitamins dance with minerals on a plate of healthful harmony. Energy sparkles in the symphony of food melody, where carrots and apples sing in a choir of wellness tunes.",
    ]]
]

data = [
    [["Discuss the role of exercise in maintaining health."],
     ["Regular exercise is fundamental to maintaining health. It strengthens the cardiovascular system, improves muscle tone and flexibility, and helps regulate weight. Exercise also releases endorphins, enhancing mental health by reducing stress and anxiety. It boosts immune function, decreases the risk of chronic diseases, and promotes overall longevity and quality of life.",
      "Doing exercise is good because it's better than sitting. You can run or walk, just move around more. It's probably good for health because moving is better than just watching TV or something.",
      "When considering exercise, it's notable that ancient Roman gladiators had rigorous training routines. Their regimen, though primarily for combat, included various physical activities. However, this historical aspect, while intriguing, doesn't directly relate to modern exercise practices or their specific health benefits.",
      "Exercise is like a rainbow of energetic butterflies fluttering in the wellness sky. Muscle marathons race with heartbeat drums, creating a symphony of sweat and joy. It's a dance of vitality where sneakers echo on the treadmill of happiness.",
     ]]
]

data = [
    [["Explain the significance of renewable energy sources."],
     ["Renewable energy sources, such as solar, wind, and hydro, are vital for sustainable development. They provide a clean, inexhaustible supply of energy, reducing reliance on fossil fuels, which are finite and pollute the environment. The use of renewable energy mitigates climate change, conserves natural resources, and promotes energy security and independence.",
      "Renewable energy is using stuff like the sun or wind for power. It's good because it's not like oil or coal, which are bad, I think. So, using the sun and wind is probably a better idea.",
      "Reflecting on renewable energy, it's interesting to ponder the windmill designs of the Dutch Golden Age. These windmills, iconic and historic, were used for various industrial purposes. However, this reflection on historical engineering doesn't directly address the modern environmental and economic impacts of renewable energy.",
      "Renewable energy is a carousel of sunbeams and breezy whispers, dancing in a green energy waltz. Solar panels bask in the glow of nature's smile, while wind turbines whirl in the symphony of eco-friendly dreams.",
    ]]
]


data = [
    [["Describe the effects of global warming."],
     ["Global warming, caused by the increased concentration of greenhouse gases from human activities, leads to significant environmental changes. It results in rising global temperatures, melting polar ice caps, and rising sea levels. These changes cause extreme weather patterns, biodiversity loss, and habitat destruction. Additionally, global warming has profound impacts on agriculture, water resources, and human health, necessitating urgent action to mitigate its effects.",
      "Global warming is when it gets hotter everywhere. It's bad because summers are really hot, and ice melts at the poles. I think it's not good for animals either, especially the ones in cold places.",
      "Considering global warming, it's fascinating to explore the fashion trends of the Victorian era, characterized by heavy fabrics and elaborate designs. Although this period's fashion is an intriguing aspect of history, it does not directly relate to the environmental and societal impacts of global warming in the contemporary context.",
      "Global warming is like a blanket of invisible sunbeams, wrapping the Earth in a simmering hug of temperature tango. Icebergs waltz in the warm ocean ballet, and polar bears conduct the symphony of climate change serenades.",
     ]
    ]
]

data


[[['Describe the effects of global warming.'],
  ['Global warming, caused by the increased concentration of greenhouse gases from human activities, leads to significant environmental changes. It results in rising global temperatures, melting polar ice caps, and rising sea levels. These changes cause extreme weather patterns, biodiversity loss, and habitat destruction. Additionally, global warming has profound impacts on agriculture, water resources, and human health, necessitating urgent action to mitigate its effects.',
   "Global warming is when it gets hotter everywhere. It's bad because summers are really hot, and ice melts at the poles. I think it's not good for animals either, especially the ones in cold places.",
   "Considering global warming, it's fascinating to explore the fashion trends of the Victorian era, characterized by heavy fabrics and elaborate designs. Although this period's fashion is an intriguing aspect of history, it does not directly relate to the environmental

In [8]:
# Now convert the DataFrame to LaTeX
latex_table = mse_vs_logistic_df.to_latex(index=False, float_format="{:0.4f}".format, multirow=True, multicolumn=True, multicolumn_format='c', escape=False)



print(latex_table)

\begin{tabular}{lrrrrrr}
\toprule
method & \multicolumn{2}{c}{Below 0.4} & \multicolumn{2}{c}{Above 0.9} & \multicolumn{2}{c}{Between 0.4 and 0.9} \\
 & mse & mae & mse & mae & mse & mae \\
\midrule
abs_sig_mse_s_075_quality & 0.0618 & 0.1981 & 0.0964 & 0.2800 & 0.0423 & 0.1652 \\
abs_logistic_over_and_under_s_75_quality & 0.0611 & 0.1999 & 0.1027 & 0.2923 & 0.0415 & 0.1662 \\
abs_logistic_aug_and_under_quality & 0.0853 & 0.2446 & 0.0786 & 0.2493 & 0.0326 & 0.1455 \\
abs_sig_mse_aug_and_under_quality & 0.1056 & 0.2827 & 0.0593 & 0.2132 & 0.0299 & 0.1360 \\
abs_sig_mse_aug_over_and_un_till16_w14_for_04_09_quality & 0.1223 & 0.3030 & 0.0441 & 0.1796 & 0.0255 & 0.1262 \\
abs_logistic_aug_over_and_un_till16_w14_for_04_09_quality & 0.0889 & 0.2555 & 0.0645 & 0.2320 & 0.0263 & 0.1301 \\
\bottomrule
\end{tabular}

